In [1]:
import numpy as np
#
# The function in this file follow closely after the MATLAB scripts
# written by Alessandro Foi and Markku Makitalo - Tampere University
# of Technology - 2011-2012
#
#    http://www.cs.tut.fi/~foi/invansc/
#
# that accompanies their paper:
#
#    Reference: M. Makitalo and A. Foi, "Optimal inversion of the
#    generalized Anscombe transformation for Poisson-Gaussian noise",
#    IEEE Trans. Image Process., doi:10.1109/TIP.2012.2202675


#-------- Variance stabilizing transforms -----------

def anscombe(x):
    '''
    Compute the anscombe variance stabilizing transform.
      the input   x   is noisy Poisson-distributed data
      the output  fx  has variance approximately equal to 1.
    Reference: Anscombe, F. J. (1948), "The transformation of Poisson,
    binomial and negative-binomial data", Biometrika 35 (3-4): 246-254
    '''
    return 2.0*np.sqrt(x + 3.0/8.0)

def inverse_anscombe(z):
    '''
    Compute the inverse transform using an approximation of the exact
    unbiased inverse.
    Reference: Makitalo, M., & Foi, A. (2011). A closed-form
    approximation of the exact unbiased inverse of the Anscombe
    variance-stabilizing transformation. Image Processing.
    '''
    #return (z/2.0)**2 - 3.0/8.0
    return (1.0/4.0 * np.power(z, 2) +
            1.0/4.0 * np.sqrt(3.0/2.0) * np.power(z, -1.0) -
            11.0/8.0 * np.power(z, -2.0) + 
            5.0/8.0 * np.sqrt(3.0/2.0) * np.power(z, -3.0) - 1.0 / 8.0)

def generalized_anscombe(x, mu, sigma, gain=1.0):
    '''
    Compute the generalized anscombe variance stabilizing transform,
    which assumes that the data provided to it is a mixture of poisson
    and gaussian noise.
    The input signal  z  is assumed to follow the Poisson-Gaussian noise model
        x = gain * p + n
    where gain is the camera gain and mu and sigma are the read noise
    mean and standard deviation.
    We assume that x contains only positive values.  Values that are
    less than or equal to 0 are ignored by the transform.
    Note, this transform will show some bias for counts less than
    about 20.
    '''
    y = gain*x + (gain**2)*3.0/8.0 + sigma**2 - gain*mu

    # Clamp to zero before taking the square root.
    return (2.0/gain)*np.sqrt(np.maximum(y, 0.0))

def inverse_generalized_anscombe(x, mu, sigma, gain=1.0):
    '''
    Applies the closed-form approximation of the exact unbiased
    inverse of Generalized Anscombe variance-stabilizing
    transformation.
    The input signal x is transform back into a Poisson random variable
    based on the assumption that the original signal from which it was
    derived follows the Poisson-Gaussian noise model:
        x = gain * p + n
    where gain is the camera gain and mu and sigma are the read noise
    mean and standard deviation.
    Roference: M. Makitalo and A. Foi, "Optimal inversion of the
    generalized Anscombe transformation for Poisson-Gaussian noise",
    IEEE Trans. Image Process., doi:10.1109/TIP.2012.2202675
    '''
    test = np.maximum(x, 1.0)
    exact_inverse = ( np.power(test/2.0, 2.0) +
                      1.0/4.0 * np.sqrt(3.0/2.0)*np.power(test, -1.0) -
                      11.0/8.0 * np.power(test, -2.0) +
                      5.0/8.0 * np.sqrt(3.0/2.0) * np.power(test, -3.0) -
                      1.0/8.0 - np.power(sigma, 2) )
    exact_inverse = np.maximum(0.0, exact_inverse)
    exact_inverse *= gain
    exact_inverse += mu
    exact_inverse[np.where(exact_inverse != exact_inverse)] = 0.0
    return exact_inverse

#def inverse_generalized_anscombe(y,mu,sigma,gain=1.0):
#    return (1.0/gain)*(gain*y/2.0)**2 - gain*3.0/8.0 - (sigma**2)/gain + mu

In [2]:
from bm3d import bm3d, BM3DProfile
import numpy as np
import scipy.misc
from matplotlib import pyplot as plt
from tifffile import imread, imsave

In [4]:
path='/Users/prakash/Downloads/Mouse skull nuclei/'

# Load the test data
noisy=imread(path+"example2_digital_offset300.tif")[:,:,:256].astype(np.float32)
# We are loading only a sub image to spped up computation

# We estimate the ground truth by averaging.
clean=np.mean(noisy[:,...],axis=0)[np.newaxis,...]

print(noisy.shape)
print(clean.shape)


(200, 512, 256)
(1, 512, 256)


In [5]:
def PSNR(gt, img, rangePSNR):
    '''
    Compute PSNR.
    Parameters
    ----------
    gt: array
        Ground truth image.
    img: array
        Predicted image.''
    '''
    mse = np.mean(np.square(gt - img))
    return 20 * np.log10(rangePSNR) - 10 * np.log10(mse)

In [6]:
offset=0
from tqdm import tqdm
psnrs=[]

for img_idx in range(noisy.shape[0]):
    print("--------------------imgae:", img_idx, " ------------------------------------")
    denoisedImgs=[]
    mses=[]
    bestMSE = None
    bestImg = None
    bestSig= None
    
    for s in tqdm(range(7)):
        sig = (s+1)*2.5
#         sig = 2*2.5

        img=noisy[img_idx,...]
        cl= clean[0]

        
        ans=anscombe(img-offset)
        denoised = inverse_anscombe (bm3d (ans,sig)) + offset


        mse = np.mean((cl-denoised)**2)
        mses.append(mse)
        denoisedImgs.append(denoised.copy())

        if bestMSE is None or (mse<bestMSE):
            bestImg = denoised.copy()
            bestMSE = mse
            bestSig = sig
        print (sig, bestSig, mse) 
    rangePSNR = np.max(cl)-np.min(cl)
    psnr = PSNR(cl, bestImg, rangePSNR)
    psnrs.append(psnr)
    print("image:", img_idx, "\t PSNR:", psnr, "\t mean PSNR:", np.mean(psnrs))


--------------------imgae: 0  ------------------------------------


 14%|█▍        | 1/7 [00:02<00:14,  2.35s/it]

2.5 2.5 228956.2279865034


 29%|██▊       | 2/7 [00:04<00:11,  2.40s/it]

5.0 5.0 125590.27529162692


...

12.5 10.0 44855.8837365286


 86%|████████▌ | 6/7 [00:17<00:02,  2.94s/it]

15.0 10.0 52080.03690340358


100%|██████████| 7/7 [00:19<00:00,  2.81s/it]

17.5 10.0 60290.23340409831
image: 199 	 PSNR: 36.060172251477944 	 mean PSNR: 36.2092610767403
